# 在MINST数据集上测试CSE、CSA、ADV, NORMAL

攻击方法： 
- FGSM
- Deepfool
- PGD

In [1]:
import torch 
import numpy as np
import os
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import sys
import copy
from torch.autograd.gradcheck import zero_gradients
from utils import *

In [2]:
# 一般的对抗模型模型
model_ADV = LeNet()
model_ADV.load_state_dict(torch.load('../model/LeNet_FASHION_MNIST_adv_e0.3.pt'))

model_NORMAL = LeNet()
model_NORMAL.load_state_dict(torch.load('../model/LeNet_FashionMnist.pt'))

<All keys matched successfully>

# clean 

In [3]:
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
data_home = 'F:\\work'
NORMALIZE = True
batch_size = 64
constant = 10

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])

test_set = torchvision.datasets.FashionMNIST(root=os.path.join(data_home, 'dataset/Fashion_mnist'), train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)


criterion = nn.CrossEntropyLoss().to(DEVICE)

In [4]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_attack = 0.3
epsilon_model = 0.3
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/' + 'LeNet_FASHION_MNIST_adv_e'+str(epsilon_model)+'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label, constant)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_CSA(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_FASHION_MNIST_cost_sensitive_extension_' + str(i_label) +'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,5)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_CSE(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_ADV(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model_NORMAL(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    
    
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.84
 1000
 1 correct: 0.983
 1000
 2 correct: 0.845
 1000
 3 correct: 0.877
 1000
 4 correct: 0.853
 1000
 5 correct: 0.975
 1000
 6 correct: 0.767
 1000
 7 correct: 0.968
 1000
 8 correct: 0.985
 1000
 9 correct: 0.959
对 model_CSE 的评估
load model for initi

 8 correct: 0.984
 1000
 9 correct: 0.949
load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 9
load cost matric: 
tensor([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.84
 1000
 1 correct: 0.983
 1000
 2 correct: 0.845
 1000
 3 correct: 0.877
 1000
 4 correct: 0.853
 1000
 5 correct: 0.975
 1000
 6 correct: 0.767
 1000
 7 correct: 0.968
 1000
 8 correct: 0.985
 1000
 9 correct: 

## 保存

In [5]:
import pandas as pd
output_dirs = './output/FASHION_MNIST'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'FASHION_MNIST_clean.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.9052
I of CSE: 0.8343
I of ADV: 0.9052
I of NORMAL: 0.9051


# FGSM 攻击

In [6]:
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
data_home = 'F:\\work'
NORMALIZE = True
batch_size = 64
constant = 10

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])

test_set = torchvision.datasets.FashionMNIST(root=os.path.join(data_home, 'dataset/Fashion_mnist'), train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)


criterion = nn.CrossEntropyLoss().to(DEVICE)

In [8]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_attack = 0.3
epsilon_model = 0.3
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/' + 'LeNet_FASHION_MNIST_adv_e'+str(epsilon_model)+'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label, constant)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_CSA, criterion_CSA, epsilon_attack, DEVICE, C)
            output = model_CSA(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_FASHION_MNIST_cost_sensitive_extension_' + str(i_label) +'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,5)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_CSE, criterion_CSE, epsilon_attack, DEVICE, C)
            output = model_CSE(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_ADV, criterion, epsilon_attack, DEVICE)
            output = model_ADV(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data, sign = my_fgsm(data, target, model_NORMAL, criterion, epsilon_attack, DEVICE)
            output = model_NORMAL(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    
    
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.847
 1000
 1 correct: 0.96
 1000
 2 correct: 0.836
 1000
 3 correct: 0.871
 1000
 4 correct: 0.85
 1000
 5 correct: 0.943
 1000
 6 correct: 0.779
 1000
 7 correct: 0.966
 1000
 8 correct: 0.965
 1000
 9 correct: 0.936
对 model_CSE 的评估
load model for initia

 8 correct: 0.961
 1000
 9 correct: 0.936
对 model_CSE 的评估
load model for initialization: ../model/LeNet_FASHION_MNIST_cost_sensitive_extension_4.pt
 1000
 0 correct: 0.77
 1000
 1 correct: 0.924
 1000
 2 correct: 0.528
 1000
 3 correct: 0.776
 1000
 4 correct: 0.636
 1000
 5 correct: 0.827
 1000
 6 correct: 0.087
 1000
 7 correct: 0.776
 1000
 8 correct: 0.854
 1000
 9 correct: 0.902
对 model_ADV 的评估
 1000
 0 correct: 0.847
 1000
 1 correct: 0.961
 1000
 2 correct: 0.846
 1000
 3 correct: 0.866
 1000
 4 correct: 0.852
 1000
 5 correct: 0.947
 1000
 6 correct: 0.77
 1000
 7 correct: 0.966
 1000
 8 correct: 0.961
 1000
 9 correct: 0.937
对 model_NORMAL 的评估
 1000
 0 correct: 0.01
 1000
 1 correct: 0.082
 1000
 2 correct: 0.005
 1000
 3 correct: 0.093
 1000
 4 correct: 0.01
 1000
 5 correct: 0.093
 1000
 6 correct: 0.0
 1000
 7 correct: 0.0
 1000
 8 correct: 0.247
 1000
 9 correct: 0.054
load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 5
load cost matric

## 保存

In [9]:
import pandas as pd
output_dirs = './output/FASHION_MNIST'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'FASHION_MNIST_FGSM.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.8953
I of CSE: 0.7123999999999999
I of ADV: 0.8953
I of NORMAL: 0.05940000000000001


# DeepFool 攻击

In [3]:
def deepfool(image, net, num_classes=10, overshoot=0.02, max_iter=3):

    """
       :param image: Image of size HxWx3
       :param net: network (input: images, output: values of activation **BEFORE** softmax).
       :param num_classes: num_classes (limits the number of classes to test against, by default = 10)
       :param overshoot: used as a termination criterion to prevent vanishing updates (default = 0.02).
       :param max_iter: maximum number of iterations for deepfool (default = 50)
       :return: minimal perturbation that fools the classifier, number of iterations that it required, new estimated_label and perturbed image
    """
    is_cuda = torch.cuda.is_available()

    if is_cuda:
#         print("Using GPU")
        image = image.cuda()
        net = net.cuda()
    else:
        pass
#         print("Using CPU")


    f_image = net.forward(Variable(image[None, :, :, :], requires_grad=True)).data.cpu().numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]

    I = I[0:num_classes]
    label = I[0]

    input_shape = image.cpu().numpy().shape
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0

    x = Variable(pert_image[None, :], requires_grad=True)
    fs = net.forward(x)
    fs_list = [fs[0,I[k]] for k in range(num_classes)]
    k_i = label

    while k_i == label and loop_i < max_iter:

        pert = np.inf
        fs[0, I[0]].backward(retain_graph=True)
        grad_orig = x.grad.data.cpu().numpy().copy()

        for k in range(1, num_classes):
            zero_gradients(x)

            fs[0, I[k]].backward(retain_graph=True)
            cur_grad = x.grad.data.cpu().numpy().copy()

            # set new w_k and new f_k
            w_k = cur_grad - grad_orig
            f_k = (fs[0, I[k]] - fs[0, I[0]]).data.cpu().numpy()

            pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

            # determine which w_k to use
            if pert_k < pert:
                pert = pert_k
                w = w_k

        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)

        if is_cuda:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).cuda()
        else:
            pert_image = image + (1+overshoot)*torch.from_numpy(r_tot)

        x = Variable(pert_image, requires_grad=True)
        fs = net.forward(x)
        k_i = np.argmax(fs.data.cpu().numpy().flatten())

        loop_i += 1

    r_tot = (1+overshoot)*r_tot

    return r_tot, loop_i, label, k_i, pert_image



In [4]:
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
data_home = 'F:\\work'
NORMALIZE = True
batch_size = 1
constant = 10

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])

test_set = torchvision.datasets.FashionMNIST(root=os.path.join(data_home, 'dataset/Fashion_mnist'), train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)


criterion = nn.CrossEntropyLoss().to(DEVICE)

In [5]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_attack = 0.3
epsilon_model = 0.3
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/' + 'LeNet_FASHION_MNIST_adv_e'+str(epsilon_model)+'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label, constant)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(1,28,28)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_CSA)
            output = model_CSA(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_FASHION_MNIST_cost_sensitive_extension_' + str(i_label) +'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,5)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(1,28,28)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_CSE)
            output = model_CSE(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(1,28,28)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_ADV)
            output = model_ADV(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = data.reshape(1,28,28)
            r, loop_i, label_orig, label_pert, pert_image = deepfool(data, model_NORMAL)
            output = model_NORMAL(pert_image)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    
    
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.601
 1000
 1 correct: 0.965
 1000
 2 correct: 0.299
 1000
 3 correct: 0.794
 1000
 4 correct: 0.57
 1000
 5 correct: 0.899
 1000
 6 correct: 0.592
 1000
 7 correct: 0.797
 1000
 8 correct: 0.958
 1000
 9 correct: 0.867
对 model_CSE 的评估
load model for initi

 1000
 9 correct: 0.252
load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 9
load cost matric: 
tensor([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1., 10.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0., 10.],
        [10., 10., 10., 10., 10., 10., 10., 10., 10.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.601
 1000
 1 correct: 0.965
 1000
 2 correct: 0.299
 1000
 3 correct: 0.794
 1000
 4 correct: 0.57
 1000
 5 correct: 0.899
 1000
 6 correct: 0.592
 1000
 7 correct: 0.797
 1000
 8 correct: 0.958
 1000
 9 correct: 0.867
对 model_CSE 

## 保存

In [6]:
import pandas as pd
output_dirs = './output/FASHION_MNIST'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'FASHION_MNIST_deepfool.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.7342
I of CSE: 0.8004
I of ADV: 0.7342
I of NORMAL: 0.19790000000000002


# PGD 攻击

In [7]:
def pgd_attack(model, images, labels, eps=0.3, alpha=2/255, iters=40, loss=None, device=None, C=None):
    if loss==None:
        loss = nn.CrossEntropyLoss()
        
    ori_images = images.data
    
    for i in range(iters) :  
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        if C==None:
            cost = loss(outputs, labels).to(device)
        else:
            cost = loss(outputs, labels, C)
        cost.backward()
        adv_images = images + alpha*images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
#         images = torch.clamp(ori_images + eta, min=0, max=1).detach_()
        images = (ori_images + eta).detach_()
        images = torch.clamp(ori_images + eta, min=-0.4242, max=2.8214).detach_()
    
    return images

In [8]:
DEVICE = torch.device(cuda_num if torch.cuda.is_available() else "cpu")
data_home = 'F:\\work'
NORMALIZE = True
batch_size = 64
constant = 10

if NORMALIZE:
    train_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
    test_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.2862,), (0.3204,))])
else:
    train_transform = transforms.Compose([transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.ToTensor()])

test_set = torchvision.datasets.FashionMNIST(root=os.path.join(data_home, 'dataset/Fashion_mnist'), train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)


criterion = nn.CrossEntropyLoss().to(DEVICE)

In [9]:
# 记录结果
results_infos = {}

# 先读取未经过对抗训练的模型
# 在进行对抗训练

# 参数
epsilon_attack = 0.3
epsilon_model = 0.3
criterion_CSA = Loss_cost_sensitive()


# 循环 对每一个类分别进行保护
for i_label in range(10):
    ################################
    # 读取模型
    model_CSA = LeNet()
    path_model_CSA = '../model/' + 'LeNet_FASHION_MNIST_adv_e'+str(epsilon_model)+'.pt'
    model_CSA.load_state_dict(torch.load(path_model_CSA))
    print('load model for initialization: {}'.format(path_model_CSA))
    model_CSA = model_CSA.to(DEVICE)
    
    
    C = get_cost_matric(i_label, constant)
    C = C.to(DEVICE)
    print('protect label: {}'.format(i_label))
    print('load cost matric: ')
    print(C)
    LABEL = 'Protect Label ' + str(i_label)

    ## 对 model_CSA 的评估
    print('对 model_CSA 的评估')
    results_info = {}
    model_CSA.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_CSA, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion_CSA, C=C)
            output = model_CSA(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[1] = images_targets
    
##################################
    ## 对 model_CSE 的评估
    print('对 model_CSE 的评估')
    # 读取模型
    model_CSE = LeNet()
    path_model_CSE = '../model/LeNet_FASHION_MNIST_cost_sensitive_extension_' + str(i_label) +'.pt'
    model_CSE.load_state_dict(torch.load(path_model_CSE))
    print('load model for initialization: {}'.format(path_model_CSE))
    model_CSE = model_CSE.to(DEVICE)
    criterion_CSE = Loss_CSE(model_CSE,1,1,5)

    model_CSE.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_CSE, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion_CSE, C=C)
            output = model_CSE(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[2] = images_targets
    
##################################
    ## 对 model_ADV 的评估
    print('对 model_ADV 的评估')


    model_ADV.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_ADV, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion)
            output = model_ADV(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[3] = images_targets
    
##################################
    ## 对 model_NORMAL 的评估
    print('对 model_NORMAL 的评估')

    model_NORMAL.eval()
    images_targets = {}
    for special_index in range(10):
        count = 0
        correct = 0

        for data, target in test_loader:
            data = data[target==special_index]
            target = target[target==special_index]
            if len(data) == 0:
                continue

            data, target = data.to(DEVICE), target.to(DEVICE)
            data = pgd_attack(model_NORMAL, data, target, eps=0.3, alpha=0.3, iters=40, loss=criterion)
            output = model_NORMAL(data)

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            count += len(data)
            print('\r {}'.format(count), end='')
        images_targets[special_index] = [count, correct/count]
        print('\n {} correct: {}'.format(special_index,correct/count))
    results_info[4] = images_targets 
    
    
    # 记录结果
    results_infos[i_label] = results_info

load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 0
load cost matric: 
tensor([[ 0., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
        [10.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [10.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.]])
对 model_CSA 的评估
 1000
 0 correct: 0.066
 1000
 1 correct: 0.347
 1000
 2 correct: 0.098
 1000
 3 correct: 0.247
 1000
 4 correct: 0.095
 1000
 5 correct: 0.128
 1000
 6 correct: 0.03
 1000
 7 correct: 0.065
 1000
 8 correct: 0.593
 1000
 9 correct: 0.202
对 model_CSE 的评估
load model for initi

 8 correct: 0.564
 1000
 9 correct: 0.193
对 model_CSE 的评估
load model for initialization: ../model/LeNet_FASHION_MNIST_cost_sensitive_extension_4.pt
 1000
 0 correct: 0.477
 1000
 1 correct: 0.881
 1000
 2 correct: 0.463
 1000
 3 correct: 0.602
 1000
 4 correct: 0.344
 1000
 5 correct: 0.169
 1000
 6 correct: 0.384
 1000
 7 correct: 0.173
 1000
 8 correct: 0.966
 1000
 9 correct: 0.746
对 model_ADV 的评估
 1000
 0 correct: 0.065
 1000
 1 correct: 0.33
 1000
 2 correct: 0.08
 1000
 3 correct: 0.204
 1000
 4 correct: 0.095
 1000
 5 correct: 0.122
 1000
 6 correct: 0.016
 1000
 7 correct: 0.048
 1000
 8 correct: 0.544
 1000
 9 correct: 0.189
对 model_NORMAL 的评估
 1000
 0 correct: 0.0
 1000
 1 correct: 0.453
 1000
 2 correct: 0.0
 1000
 3 correct: 0.012
 1000
 4 correct: 0.0
 1000
 5 correct: 0.304
 1000
 6 correct: 0.002
 1000
 7 correct: 0.008
 1000
 8 correct: 0.03
 1000
 9 correct: 0.078
load model for initialization: ../model/LeNet_FASHION_MNIST_adv_e0.3.pt
protect label: 5
load cost matric:

## 保存

In [10]:
import pandas as pd
output_dirs = './output/FASHION_MNIST'
if os.path.exists(output_dirs) is False:
    os.makedirs(output_dirs)

I_avg = {'CSA':[], 'CSE':[], 'ADV':[], 'NORMAL':[]}
writer = pd.ExcelWriter(os.path.join(output_dirs, 'FASHION_MNIST_PDG.xlsx'))
for i in results_infos.keys():
    tmp = results_infos[i]
    
    I_avg['CSA'].append(tmp[1][i][1])
    I_avg['CSE'].append(tmp[2][i][1])
    I_avg['ADV'].append(tmp[3][i][1])
    I_avg['NORMAL'].append(tmp[4][i][1])
    
    df  = pd.DataFrame(tmp)
    df.columns = ['CSA', 'CSE', 'ADV', 'NORMAL']
    df = pd.DataFrame([df[i].apply(lambda x: x[1]) for i in df.columns])
    df = df.sort_index()
    
    df.to_excel(writer, sheet_name=str(i))
writer.save()

print('I of CSA: {}'.format(np.mean(I_avg['CSA'])))
print('I of CSE: {}'.format(np.mean(I_avg['CSE'])))
print('I of ADV: {}'.format(np.mean(I_avg['ADV'])))
print('I of NORMAL: {}'.format(np.mean(I_avg['NORMAL'])))

I of CSA: 0.16970000000000002
I of CSE: 0.4866
I of ADV: 0.17020000000000002
I of NORMAL: 0.0885
